In [1]:
import os
import re
import pandas as pd
def convert_ascii_to_csv(input_file, output_file=None):
    """
    Mengkonversi file ASCII dari data buoy RAMA menjadi format CSV.
    
    Parameters:
    input_file (str): Path ke file ASCII
    output_file (str, optional): Path untuk menyimpan file CSV hasil. 
                                 Jika None, nama file akan sama dengan input dengan ekstensi .csv
    
    Returns:
    str: Path ke file CSV yang dihasilkan
    """
    # Jika output_file tidak ditentukan, gunakan nama input dengan ekstensi .csv
    if output_file is None:
        output_file = os.path.splitext(input_file)[0] + '.csv'
    
    # Baca seluruh isi file
    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    # Identifikasi header kolom dan data
    data_rows = []
    current_headers = None
    
    for line in lines:
        # Abaikan baris yang berisi informasi lokasi, unit, time, depth
        if re.match(r'^\s*Location:|^\s*Units:|^\s*Time:|^\s*Depth', line):
            continue
        
        # Identifikasi header kolom (selalu berisi YYYYMMDD dan HHMM)
        if 'YYYYMMDD' in line and 'HHMM' in line:
            current_headers = line.strip().split()
            continue
        
        # Identifikasi baris data (selalu dimulai dengan tahun, 4 digit angka pertama)
        if re.match(r'^\s*\d{8}\s', line):
            data_values = line.strip().split()
            # Pastikan hanya baris data valid yang diambil
            if len(data_values) >= len(current_headers):
                data_rows.append(data_values[:len(current_headers)])
    
    # Buat DataFrame dari data yang ditemukan
    df = pd.DataFrame(data_rows, columns=current_headers)
    
    # Gabungkan kolom tanggal dan waktu
    df['Timestamp'] = pd.to_datetime(df['YYYYMMDD'] + ' ' + df['HHMM'], format='%Y%m%d %H%M')
    
    # Hapus kolom asli tanggal dan waktu
    df = df.drop(['YYYYMMDD', 'HHMM'], axis=1)
    
    # Konversi nilai numerik
    for col in df.columns:
        if col != 'Timestamp':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Ganti nilai -9.99 dengan NaN
    df = df.replace(-9.99, pd.NA)
    
    # Simpan ke CSV
    df.to_csv(output_file, index=False)
    
    return output_file
def process_multiple_files(input_directory, output_directory=None, file_pattern='*.ascii'):
    """
    Memproses multiple file ASCII dalam satu direktori
    
    Parameters:
    input_directory (str): Direktori yang berisi file ASCII
    output_directory (str, optional): Direktori untuk menyimpan file CSV, jika None akan disimpan di direktori yang sama
    file_pattern (str): Pola file yang akan diproses, default '*.ascii'
    
    Returns:
    list: Daftar file CSV yang dihasilkan
    """
    
    if output_directory is None:
        output_directory = input_directory
    
    # Pastikan output_directory ada
    os.makedirs(output_directory, exist_ok=True)
    
    processed_files = []
    
    # Cari semua file yang sesuai pola
    for input_file in glob.glob(os.path.join(input_directory, file_pattern)):
        filename = os.path.basename(input_file)
        output_file = os.path.join(output_directory, os.path.splitext(filename)[0] + '.csv')
        
        print(f"Memproses {filename}...")
        convert_ascii_to_csv(input_file, output_file)
        processed_files.append(output_file)
    
    return processed_files

# Contoh penggunaan
if __name__ == "__main__":
    # Untuk satu file
    convert_ascii_to_csv('/run/media/cryptedlm/localdisk/Kuliah/Tugas Akhir/Dataset/Buoys/0N90E/ASCII/rad0n90e_dy.ascii')
    
    # Untuk banyak file
    #process_multiple_files('/path/to/data/directory')